In [ ]:
# Module 2 - Libraries

import os
import sys
import cv2
import time
import glob
import pubnub
import smtplib
import numpy as np
import pytesseract
from PIL import Image
import tensorflow as tf
from datetime import datetime
from cryptography.fernet import Fernet
from pubnub.enums import PNStatusCategory
from pubnub.exceptions import PubNubException 
from pubnub.callbacks import SubscribeCallback
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub, SubscribeListener, SubscribeCallback, PNStatusCategory

import matplotlib.pyplot as plt
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')  

import warnings
warnings.filterwarnings('ignore')
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

database = {}

In [ ]:
# Module 2 Execution

def main():
    pnconf = PNConfiguration()   
    pnconf.publish_key = 'pub-c-47de1ea1-b0ba-462c-ba6a-1c4fa5371251'
    pnconf.subscribe_key = 'sub-c-d5e026e5-08a6-401d-9651-a1ba89559278'
    pnconf.uuid = 'Project'
    channel = 'Project'
    pubnub = PubNub(pnconf)
    pubnub.add_listener(VFDSubscribeCallback())
    pubnub.subscribe().channels(channel).execute()

In [ ]:
class VFDSubscribeCallback(SubscribeCallback):
    def message(self, pubnub, event):
        print(event.message)
        remoteCommand = event.message["command"]
        channel = 'Project'
        
        if remoteCommand == "Identify":         
            # Image Capturing
            camera = cv2.VideoCapture(0)
            time.sleep(3)
            return_value, image = camera.read()
            cv2.imwrite('capturing.png', image)
            del(camera)
            cv2.destroyAllWindows()
            Image_loc = ["/Users/jai/CPS_Image/capturing.png"]
            Labels_loc ='/Users/jai/CPS_Image/mscoco_label_map.pbtxt'
            
            # Loading the Predefined Model and Building a Detection Method
            Predefined_Model ='/Users/jai/CPS_Image/'
            detect_fn = tf.saved_model.load(Predefined_Model)
            detect_fn = detect_fn.signatures['serving_default']
            category_index = label_map_util.create_category_index_from_labelmap(Labels_loc,use_display_name=True)

            def load_image_into_numpy_array(path):
                return np.array(Image.open(path))

            for image_path in Image_loc:
                image_np = load_image_into_numpy_array(image_path)
                input_tensor = tf.convert_to_tensor(image_np)
                input_tensor = input_tensor[tf.newaxis, ...]
                detections = detect_fn(input_tensor)
                num_detections = int(detections.pop('num_detections'))
                detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
                detections['num_detections'] = num_detections
                detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

                viz_utils.visualize_boxes_and_labels_on_image_array(
                      image_np,
                      detections['detection_boxes'],
                      detections['detection_classes'],
                      detections['detection_scores'],
                      category_index,
                      use_normalized_coordinates=True,
                      max_boxes_to_draw=400,
                      min_score_thresh=.30,
                      agnostic_mode=False)
                plt.figure(figsize=(12,12))
                plt.imshow(image_np)
                plt.savefig('EMG {0}.jpg')
            plt.show()
            
            for i in detections['detection_classes']:
                print(category_index[i]['name'])
                pubnub.publish().channel(channel).message(category_index[i]['name']).sync() 
            data = "Do you want to Detect the Number Plate?"
            pubnub.publish().channel(channel).message(data).sync()
            main()
            
        elif remoteCommand == "Yes":
            cam = cv2.VideoCapture(0)
            time.sleep(3)
            return_value, image = cam.read()
            cv2.imwrite('plate.png', image)
            del(cam)
            cv2.destroyAllWindows()

            img = cv2.imread('plate.png')
            img_text = pytesseract.image_to_string(img)
            text1 = "The Number plate: " +img_text
            pubnub.publish().channel(channel).message(text1).sync() 
            pubnub.publish().channel(channel).message("Encrypting the info...").sync() 

            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

            message = img_text
            key = Fernet.generate_key()
            fernet = Fernet(key)
            encMessage = fernet.encrypt(message.encode())

            database[dt_string]= encMessage

            cmd ="Informating is Stored"
            pubnub.publish().channel(channel).message(cmd).sync()
            cmd ="Do you want to Allow?"
            pubnub.publish().channel(channel).message(cmd).sync()
            main()
            
        elif remoteCommand == "No":
            data = "Access Denied !!!"
            pubnub.publish().channel(channel).message(data).sync() 
            
        
if __name__ == '__main__':
    main()